# Eco-evolutionary dynamics in agriculture: a study in crop rotations

### Packages

In [1]:
#!/usr/bin/env python

from __future__ import division
from subprocess import call
from scipy import *
from scipy.linalg import eigh as largest_eigh
from scipy.integrate import odeint
import scipy.integrate as spi
import numpy as np
from itertools import combinations
from itertools import repeat
import matplotlib.pyplot as plt
import math
import io
import csv
import sys, os
import pandas as pd
from pylab import *

### Parameters

In [2]:
## For crop rotation cash yield and soil quality

q0 = 1 # init soil quality
y0 = 0 # init cash yield
K = 2 #carrying capacity
beta_h1 = -1.5  #soil contribution h1
beta_h1_r = -1.5 #soil contribution h1_r
beta_h2 = 1 #soil contribution h2
y_h1 = 1 #cash contribution h1
y_h1_r = 0.9 #cash contribution h1_r
y_h2 = 0 #cash contribution h2



## For eco-evo dynamics
epsilon = 0.5 # pathogen clearance
k = 0.1 # cost of resistance
mu = 0.1 # transition probability
a = 0.5 # GfG proportion
sigma = 0.04 # pathogen virulence of p1
H_dens = 50 # host density
dp = 0.5 # pathogen deathrate



## Time (needed for simulation)
t_step = 0.005
t_last = int(1/t_step)


### Infection environment (pathogen yes/no, time of infection)

In [3]:
pathogen_true = int(raw_input("pathogen true? 1 = true, 0 = false")) #presence or absence of pathogen
infection_time = int(raw_input("infection time?")) # time the pathogen infects

df = pd.DataFrame() #data frame for output results
df2 = pd.DataFrame() 

pathogen true? 1 = true, 0 = false1
infection time?0


### Sequence population (create all possible patterns of a certain length)

In [4]:
l = 10  #length
num_seq = 2**l   #population size

cash = ['h1'] * l
cover = ['h2'] * l

list_seq = [] # population

def replacements(cash, cover):
    for count in range(len(cash)):
        for positions in combinations(range(len(cash)), count):
            yield [cover[i] if i in positions else name for i, name in enumerate(cash)]
    yield cover

for result in replacements(cash,cover):
    list_seq.append(result)
    
    
# t_step = 0.005
# t_last = int(1/t_step)

In [5]:
## define a binary sequence (1,0) for each crop type

def get_h_seq(seq):
    crop_types = ['h1','h2','h1_r']
    seq_elements = seq
    binaries = {k:[int(k in v) for v in seq_elements]for k in crop_types}
    h1_seq,h2_seq,h1_r_seq = binaries['h1'],binaries['h2'],binaries['h1_r']
    
    return h1_seq,h2_seq,h1_r_seq



### Transition and fitness matrices (section 2.2)

In [6]:
def transition_matrix():
    d = np.array([
    [1 - 2*mu ,mu,0,0,0,mu,0,0,0,0],
    [mu,1 - 3*mu,mu,0,0,0,mu,0,0,0],
    [0,mu,1 - 3*mu,mu,0,0,0,mu,0,0],
    [0,0,mu,1 - 3*mu,mu,0,0,0,mu,0],
    [0,0,0,mu,1 - 2*mu,0,0,0,0,mu],
    [mu,0,0,0,0,1 - 2*mu,mu,0,0,0],
    [0,mu,0,0,0,mu,1 - 3*mu,mu,0,0],
    [0,0,mu,0,0,0,mu,1 - 3*mu,mu,0],
    [0,0,0,mu,0,0,0,mu,1 - 3*mu,mu],
    [0,0,0,0,mu,0,0,0,mu,1 - 2*mu]
    ])
    # d = np.identity(10) ### in case we don't want any transition between types
    return(d)

q_matrix = transition_matrix()

In [7]:
def fitness_matrix():
    w_matrix = np.array([[1,0,0],[1.25,0,0],[1.5,0,0],[1.75,0,0],[2,0,0],
    [a*(1 - a*k),1 - a*k,0],[a*(1.25 - a*k),1.25 - a*k,0],[a*(1.5 - a*k),1.5 - a*k,0],[a*(1.75 - a*k),1.75 - a*k,0],
    [a*(2 - a*k),2 - a*k,0]])

    return(w_matrix)

w_matrix = fitness_matrix()


### Eco-Evo dynamics (section 2.2)

In [8]:
def infection(h1_seq,h1_r_seq,h2_seq):
    def HP_evol(t,init): ## equations for host-pathogen coevolution
        h = np.array(init[0:3],dtype='float16')
        p = np.array(init[3:13],dtype='float16')
        hdot = np.zeros(3)
        pdot = np.zeros(10)
        
        for h_type in range(0,3):    
            hdot[h_type] = h[h_type] * np.sum((-sigma * np.multiply(w_matrix[:,h_type], p,dtype='float16')))

        for p_type in range(0,10):
            pdot[p_type] = np.sum((np.multiply(q_matrix[:,p_type], p,dtype='float16') * np.sum((sigma * np.multiply(w_matrix[p_type,:], h,dtype='float16'))))) - p[p_type]*dp
        
        ydot = np.append(hdot,pdot)
        return(ydot)
    
    
    t_i = 0 #init time (start of season)
    t_f = 1 #final time (end of season)
    general_ts = []
    general_zs = []
    z0 = np.zeros(13)
    for i in range(0,l):

      ## intiation of host density in each season
    
      h1 = H_dens * (h1_seq[i]) 
      h1_r = H_dens * (h1_r_seq[i])
      h2 = H_dens * (h2_seq[i])
    
      
      z0[0:3] = [h1,h1_r,h2]

      if t_i == infection_time and pathogen_true == 1: ## introducing the pathogen at infection time with a density of 1
        p1 = 1
        z0[3] = p1

      ## ODE solver
    
      t_end = 1
      t_start = 0.
      t_step = 0.005
      t_interval = np.arange(t_start, t_end, t_step)

      myode =  spi.ode(HP_evol)
      myode.set_integrator('lsoda',nsteps = 500000000)
      myode.set_initial_value(z0,t_start)

      ts = [0]
      zs = [z0]  


      while myode.successful() and myode.t < t_end:
         myode.integrate(myode.t + t_step)
         ts.append(myode.t)
         zs.append(myode.y)

      z_final = zs[t_last]
      #
      h1_f = z_final[0]
      h1_r_f = z_final[1]
      h2_f = z_final[2]
      
      p = np.zeros(10)
      for n in range(0,10):
        p[n] = epsilon * z_final[(n + 3)] ## the pathogen for the next season depends on epsilon, or pathogen clearance
      
      z0[3:13] = p
  
      t_extra = np.full((t_last + 1,),i)
      general_ts.append(ts + t_extra)
      general_zs.append(zs)

      t_i = t_i + 1  #discrete time increase
      t_f = t_f + 1
    return general_ts,general_zs





### Soil quality and cash yield in crop rotation patterns (section 2.1 and 2.3)

In [11]:
def getdeltas(seq,zs):  ## section 2.3, delta as the ratio of host at the end of season and initial host density
    zs = np.array(zs)
    zz = zs[:,t_last]
    zz_all = []
    for j in range(0,l):
        zz_1 = zz[j]
        zz_2 = zz_1[0:3]
        zz_all.append(zz_2)
    zz_all = np.array(zz_all)
    x = np.sum(zz_all,axis = 1)    
    deltas = x/H_dens 
    return deltas



def fitness_function(init_cash,init_soil,seq,deltas): ## fitness of sequences
# 
    #print("deltas",deltas)

    soil_vector = []
    cash_vector = []
    n_h2 = 0 #n of consecutive cover crops
    n_h1_hr = 0 #n of consecutive cash crops
    
    soil_seq = np.zeros(l + 1) #vector that will have soil changes
    soil_seq[0] = init_soil

    cash_seq = np.zeros(l + 1) #vector that will have cash changes
    cash_seq[0] = init_cash

    for t in range(0,l):
        d = deltas[t]
        
        if seq[t] == 'h2': #soil quality and cash yield for cv crops
            n_h2 = n_h2 + 1
            n_h1_hr = 0
            soil_seq[t + 1] = max(0.01, min(1.99,((K * soil_seq[t] * math.exp(beta_h2)) / (K + soil_seq[t] * (math.exp(beta_h2) - 1)))))
            cash_seq[t + 1] = cash_seq[t] + y_h2 * d * soil_seq[t]
        elif seq[t]=='h1': 
            n_h1_hr = n_h1_hr + 1
            n_h2 = 0
            soil_seq[t + 1] = max(0.01, min(1.99,((K * soil_seq[t] * math.exp(beta_h1)) / (K + soil_seq[t] * (math.exp(beta_h1) - 1)))))
            cash_seq[t + 1] = cash_seq[t] + y_h1 * d * soil_seq[t]
        elif seq[t]=='hr': 
            n_h1_hr = n_h1_hr + 1
            n_h2 = 0
            soil_seq[t + 1] = max(0.01, min(1.99,((K * soil_seq[t] * math.exp(beta_hr)) / (K + soil_seq[t] * (math.exp(beta_hr) - 1)))))
            cash_seq[t + 1] = cash_seq[t] + y_hr * d * soil_seq[t]

    fitness = cash_seq[l]
    return cash_seq,soil_seq,fitness




for j in range(0,num_seq):
    seq = list_seq[j]
    h1_seq,h1_r_seq,h2_seq= get_h_seq(seq)
    
    ts,zs = infection(h1_seq,h1_r_seq,h2_seq)
    deltas = getdeltas(seq,zs)
    init_soil = np.full(1,q0)    #init soil is 1
    init_cash = np.full(1,y0)   #init cash is 0
    cash_seq,soil_seq,fitness = fitness_function(init_cash,init_soil,seq,deltas)
    z = np.array(zs).T

    h1,h1_r,h2,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10 = z
    
    
    p_sum1 = p1 + p2 + p3 + p4 + p5
    final_p5 = p5[-1,-1] 
    final_pathogen = p_sum1[-1,-1]    
    freq_p5 = final_p5 / final_pathogen  
    
    
    soil_seq = soil_seq[1:11]
    cash_seq = cash_seq[1:11]

    
    df['h1_seq'] = np.append(np.nan,h1_seq)
    df['h1_r_seq'] = np.append(np.nan,h1_r_seq)
    df['h2_seq'] = np.append(np.nan, h2_seq)
    df['deltas'] = np.append(np.nan,deltas)
    df['soil'] = np.append(np.nan,soil_seq)
    df['cash'] = np.append(np.nan,cash_seq)
    df['freq p5'] = np.append(h1_seq,freq_p5)
    
    df2['seq and fitness'] = np.append(h1_seq,cash_seq[9])
    
    df.to_csv('out{0}.csv'.format(j+1))
    df2.to_csv('h1_seq{0}.csv'.format(j+1), index = False)


22.5105737509
42.2727701109
0.532507656628
(10,)
()
22.5331140364
42.1815807214
0.534193210664
(10,)
()
22.8807020778
42.1815464342
0.542433931706
(10,)
()
22.9857914194
42.1073700616
0.545885230681
(10,)
()
22.8774804559
41.8274496819
0.546948968438
(10,)
()
22.4737396447
41.1400026512
0.546274628012
(10,)
()
21.7256524989
39.8335882959
0.545410379239
(10,)
()
20.3607835205
37.3992633279
0.544416699921
(10,)
()
17.7986545195
32.8550557236
0.54173259267
(10,)
()
13.2015867511
24.7963062055
0.532401344042
(10,)
()
6.32861383172
12.7959342247
0.494580053366
(10,)
()
24.2161368932
42.0520852555
0.575860548794
(10,)
()
23.1553531031
41.8644745565
0.553102680696
(10,)
()
22.8460470461
41.697724583
0.547896732366
(10,)
()
22.6598867153
41.2316765733
0.549574710478
(10,)
()
21.8997688639
39.8607333341
0.549407073882
(10,)
()
20.3920702735
37.2655992426
0.547208972562
(10,)
()
17.7702991512
32.6786884674
0.543788627532
(10,)
()


KeyboardInterrupt: 